#### Place cell identification pipeline 2.0


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.ndimage import gaussian_filter
from sklearn.feature_selection import mutual_info_regression as mi_skl
import seaborn as sns
from scipy import stats
import time
from matplotlib import gridspec
import os 

color_dic = {'PRE':'midnightblue', 
             'SAM':'lightseagreen', 
             'CHO':'deepskyblue',
             'Trajectory':'lightgrey',
             'Startbox':'red',
             'SW1':'turquoise', 
             'SW2':'mediumpurple', 
             'SW3':'cornflowerblue'} 

In [2]:
#Load data
ANIMALS = ['H0465','H0466', 'H0493', 'H2202','H2203']
#-------------------------------------------------------------------------------------------
codeDir = os.getcwd()
len2del = sum([len(i) for i in codeDir.split('/')[-2:]])+1
homeDir = codeDir[:-len2del]

dataDir = codeDir[:-sum([len(i) for i in codeDir.split('/')[-3:]])-2]
SaveTo2 = homeDir+'Figures/Figure 2 Calcium imaging' #Where to save figures

TRIALS = pd.read_csv(homeDir + "Data/Trial_breakdown.csv")
pcDIR = homeDir + "Data/Place cell meta/"

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Trial_breakdown.csv'

In [42]:
def arena_binned(x, y, xdim, ydim, pix_cm, pf_cm):
    d_x = xdim[0] - xdim[1] # xdim[0] = max, xdim[1] = min
    d_y = ydim[0] - ydim[1] # ydim[0] = max, ydim[1] = min
    nBnx = int((d_x/(pix_cm*pf_cm)))
    nBny = int((d_y/(pix_cm*pf_cm)))
    bn_x = [int(i) for i in np.linspace(xdim[0],xdim[1],nBnx)]
    bn_y = [int(i) for i in np.linspace(ydim[0], ydim[1],nBny)]

    bx = ((x-bn_x[0])/(bn_x[1]-bn_x[0])).astype(int)
    by = ((y-bn_y[0])/(bn_y[1]-bn_y[0])).astype(int)
    bx[bx>=20] = 19
    by[by>=20] = 19
    S = np.vstack((bx,by))
    linS = np.ravel_multi_index(S,(nBnx,nBny))
    occMap = sps.csr_matrix((np.ones(len(bx)),(bx,by)),shape=(nBnx,nBny),dtype=float).todense()
    return linS.astype(float), occMap, nBnx, nBny, bx, by


def bursting_check(linSpf, linS, e_trace, tr_delay):
    #Place field timestamps
    pft_idxs = [i for i in range(len(linS)) if linS[i] in linSpf]
  
    #Seperate timestamps into traverals
    tr_idxs_all = []; tr_idxs = []
    i = 0
    while i < len(pft_idxs)-1:
        tr = pft_idxs[i+1]-pft_idxs[i]
        tr_idxs.append(pft_idxs[i])

        if tr > tr_delay: #acceptable delay between traversals
            tr_idxs_all.append(tr_idxs)
            tr_idxs = []
        i += 1
    
    if len(tr_idxs_all) > 0: #If 1 single traversal
        
        #Calculate the number of traversals with events
        tr = 0
        for i in tr_idxs_all:
            n_events = sum(e_trace[i])
            if n_events > 0:
                tr += 1
    else:
        tr = 1; tr_idxs_all = [[0]]
    
    return tr,(tr/len(tr_idxs_all))*100

In [43]:
#PLACE CELL INCLUSION CRITERIA
n_events = 3 # CONDITION 1: set n events for a cell to be considered
perc_thresh = 99 # CONDITION 2: Set place cell spatial info percentile 
n_tr = 3 # CONDITION 3: set number of traversals to be considered
tr_thresh = 20 # CONDITION 4: % traversals that cell fires <-- accounts for rdm bursting

#ADDITIONAL PARAMS
xdim = [575, 175] # Set arena x dimensions (northwest - (-20,-20), 320x320)
ydim = [450,50] # Set Arena y dimensions
pix_cm = 5 # Set how many pixels = 1 cm
pf_cm = 4 # Set bin size for place field (cms)
shuffles = 5000 # set number of shuffles to create SI distribution
tr_delay = 40 # acceptable delay between traversals (frames)

In [71]:
for animal in ANIMALS[2:3]:
    animDIR = dataDir + str(animal) + '/01_Aligned_LR/'
    for file in os.listdir(animDIR):
        if file != 'traces' and file.endswith('.csv'):
            session, stage = file.split('_')[1:3]
            print(animal, session, stage)
            #Read in aligned events df
            meta = pd.read_csv(animDIR+'/'+file)

            #Remove all start box activity
            a_meta = meta[meta['position']!=12]
            a_meta = a_meta[a_meta['position']!=str(12)]

            #Only analyse moving points # INCLUDE SPEED FUNCTION HERE
            mov_meta = a_meta[a_meta['Movement status']=='moving']
            x = mov_meta['x'].values.astype(float)
            y = mov_meta['y'].values.astype(float)

            #Bin and linearize occupancy
            linS, occMap, nBnx, nBny, bx, by = arena_binned(x = x, y = y,
                                                            xdim = xdim, ydim = ydim,
                                                            pix_cm = pix_cm, pf_cm = pf_cm)

            #Isolate and binarize events
            e_traces = mov_meta[mov_meta[[col for col in mov_meta if col.startswith(' C')]]
                                .loc[:,~(mov_meta[[col for col in mov_meta if col.startswith(' C')]] == " nan").any()].columns.values]
            cell_IDS = e_traces.columns.values
            e_traces[e_traces > 0] = 1 # OPTIONAL: Binarized events
            e_traces = e_traces.values.T

            criteria = np.zeros((len(e_traces),4));
            pcs = []; Percs = []; SI =[]
            for i, ev in enumerate(e_traces):
                print(i+1,'/',len(cell_IDS))

                #Calculate place map
                rateMap = sps.csr_matrix((ev,(bx,by)),shape=(nBnx,nBny),dtype=float).todense()
                placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0

                #CONDITION 1: Exclude cells that have less than n events
                if  len([e for e in ev if e > 0]) > n_events:
                    criteria[i][0] = int(1)

                    #Calculate SI and compare to distribution
                    res_eve = list(np.array(ev.reshape(-1,1)).astype(float))
                    si = mi_skl(res_eve, linS, discrete_features=True,n_neighbors=3)
                    SI.append(si)

                    #Create distribution of SI with shuffled events
                    si_shuf = []
                    for s in range(shuffles): 
                        np.random.shuffle(res_eve)
                        si_shuf.append(mi_skl(res_eve, linS, discrete_features=True, n_neighbors=3))
                        if s > 0 and s % 1000 == 0:
                            print(s, 'shuffles completed')

                    #Calculate percentile
                    perc = stats.percentileofscore(si_shuf, si)
                    Percs.append(perc)

                    #Account for n traversalss random bursting cells
                    s_placeMap = gaussian_filter(placeMap, sigma = 1)
                    sn_placeMap = np.array(s_placeMap)/np.max(s_placeMap)
                    sn_placeMap[sn_placeMap < 0.2] = 0
                    pf_idxs = np.argwhere(sn_placeMap > 0)
                    S_pf = np.vstack((pf_idxs[:,0],pf_idxs[:,1]))
                    linSpf = np.ravel_multi_index(S_pf,(nBnx,nBny))
                    tr, rdm_burst = bursting_check(linSpf, linS, ev, tr_delay)

                    # CONDITION 2: cell SI > Threshold are considered a Place cell
                    if perc >= perc_thresh:
                        criteria[i][1] = int(1)

                    #CONDITION 3: Number of traversals required   
                    if tr > n_tr: 
                        criteria[i][2] = int(1)

                    #CONDITION 4: number of traversals with cell firing  
                    if rdm_burst >= tr_thresh: 
                        criteria[i][3] = int(1)

                    #If all place cell criteria, then place cell identified
                    if sum(criteria[i]) == 4:
                        print(cell_IDS[i], 'Is a place cell with %d traversals' %tr)
                        pcs.append(1)
                    else:
                        pcs.append(0)         
                else:
                    pcs.append(0)
                    SI.append(0)
                    Percs.append(0)
            
            #Read in animal place cell df
            pcANI = pd.read_csv(pcDIR+animal + '_place_cell_meta2.csv')
            
            #Add stage to DF
            PC_st = pd.DataFrame({'Cell IDS':cell_IDS,'Place cells':pcs,
                                  'Criteria':list(criteria),'SI':SI,
                                  'Percentile':Percs,
                                  'Place cells':pcs,
                                  'SI':SI, 
                                  'Session':session, 
                                  'Stage':stage})
            
            pcANI = pcANI.append(PC_st)
            pcANI.to_csv(pcDIR + '%s_place_cell_meta2.csv' %(animal), index=False)
            print(sum(pcs), 'out of %d cells have been identified as place cells' %len(pcs))
            print((sum(pcs)/len(pcs))*100, ' Percent of cells are place cells')

H0493 P01 PRE
1 / 89


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3713: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


2 / 89


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


3 / 89


...

 C94 Is a place cell with 6 traversals
136 / 140
137 / 140
 C96 Is a place cell with 5 traversals
138 / 140


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


139 / 140
140 / 140


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


17 out of 140 cells have been identified as place cells
12.142857142857142  Percent of cells are place cells
